#Mount and Starting

In [ ]:
!pip install fingerflow

In [ ]:
import cv2
import numpy as np

from fingerflow.extractor import constants


def preprocess_image_data(raw_image_data):
    print("Image shape:", raw_image_data.shape)
    print("Image dtype:", raw_image_data.dtype)

    image_data = np.array(cv2.cvtColor(raw_image_data, cv2.COLOR_BGR2GRAY))
    image_size = np.array(image_data.shape, dtype=np.int32) // 8 * 8
    image = image_data[:image_size[0], :image_size[1]]

    output = dict()
    output['original_image'] = image_data
    output['image_size'] = image_size
    output['image'] = image

    return output





def get_minutiae_patch_coordinate(patch_center, image_size):
    start_shift = 0
    end_shift = 0

    if patch_center < constants.PATCH_MINU_RADIO:
        start_shift = constants.PATCH_MINU_RADIO - patch_center

    if patch_center + constants.PATCH_MINU_RADIO > image_size:
        end_shift = patch_center + constants.PATCH_MINU_RADIO - image_size

    patch_start = patch_center - constants.PATCH_MINU_RADIO
    patch_end = patch_center + constants.PATCH_MINU_RADIO

    return patch_start + start_shift - end_shift, patch_end + start_shift - end_shift


def get_minutiae_patch(x, y, image):
    x_start, x_end = get_minutiae_patch_coordinate(int(x), image.shape[0])
    y_start, y_end = get_minutiae_patch_coordinate(int(y), image.shape[1])
    patch_minu = image[x_start:x_end, y_start:y_end]

    return patch_minu


def resize_minutiae_patch(minutiae_patch):
    minutiae_patch = cv2.resize(minutiae_patch, dsize=(224, 224), interpolation=cv2.INTER_NEAREST)

    ret = np.empty(constants.INPUT_SHAPE, dtype=np.uint8)

    ret[:, :, 0] = minutiae_patch
    ret[:, :, 1] = minutiae_patch
    ret[:, :, 2] = minutiae_patch

    minutiae_patch = ret
    minutiae_patch = np.expand_dims(minutiae_patch, axis=0)

    return minutiae_patch


In [ ]:
# from .minutiae_net import MinutiaeNet
from fingerflow.extractor.minutiae_net import MinutiaeNet
from fingerflow.extractor.classify_net import ClassifyNet

# from .classify_net import ClassifyNet
from fingerflow.extractor.core_net import CoreNet
from fingerflow.extractor import utils

# from .core_net import CoreNet
# from . import utils


class Extractor:
    def __init__(self, coarse_net_path, fine_net_path, classify_net_path, core_net_path):
        self.__extraction_module = MinutiaeNet(coarse_net_path, fine_net_path)
        self.__classification_module = ClassifyNet(classify_net_path)
        # self.__core_detection_module = CoreNet(core_net_path)

    def extract_minutiae(self, image_data):
        preprocessed_image = preprocess_image_data(image_data)

        extracted_points = self.__extraction_module.extract_minutiae_points(
            preprocessed_image['image'],
            preprocessed_image['original_image'])

        classified_points = self.__classification_module.classify_minutiae_points(
            preprocessed_image['original_image'],
            extracted_points)

        # core_coordinates = self.__core_detection_module.detect_fingerprint_core(image_data)

        # return {'core': core_coordinates, 'minutiae': classified_points}
        return {'minutiae': classified_points}


In [ ]:
from datetime import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow
# from /content/drive/MyDrive/fingerflow-main/src/fingerflow/extractor/extractor.py
# # from fingerflow import extractor
# from fingerflow import extractor
# from .content.drive.MyDrive.fingerflow_main.src.fingerflow.extractor.extractor import *
# from fingerflow.extractor.CoreNet import *
# from fingerflow.extractor.extractor import Extractor
# from fingerflow.extractor.extractor import *


COARSE_NET_PATH = "CoarseNet.h5"
FINE_NET_PATH = "FineNet.h5"
CLASSIFY_NET_PATH = "ClassifyNet_6_classes.h5"
CORE_NET_PATH = 'CoreNet.weights'
IMG_PATH = '102_4.tif'
RADIUS = 5

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
IMG_PATH = '102_4.tif'
image = cv2.imread(IMG_PATH)
# Display the image
import matplotlib.pyplot as plt

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()



#for a single image roi

can use this to check for the same impression

In [ ]:
def get_minutiae_color(point_class):
    switcher = {
        0: (0, 0, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")

extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

image = cv2.imread(IMG_PATH)

extracted_data = extractor.extract_minutiae(image)

extracted_minutiae = extracted_data['minutiae']
core = extracted_data['core']

start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])
color = (0, 0, 255)

cv2.rectangle(image, start_point, end_point, color, 2)


if len(extracted_minutiae) > 0:
    for index, minutia in extracted_minutiae.iterrows():
        print(minutia)
        x = minutia['x']
        y = minutia['y']
        o = minutia['angle']
        minutia_class = minutia['class']

        start_point = (int(x-RADIUS), int(y-RADIUS))
        end_point = (int(x+RADIUS), int(y+RADIUS))

        color = get_minutiae_color(int(minutia_class))

        cv2.rectangle(image, start_point, end_point, color, 2)

        line_start_point = (int(x), int(y))
        line_end_point = (int(x + 20 * np.cos(o)), int(y + 20 * np.sin(o)))

        cv2.line(image, line_start_point, line_end_point, color, 2)

plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


In [ ]:
from datetime import datetime
import cv2
import numpy as np
# from google.colab.patches import cv2_imshow

# ... (Your existing code)

# Extracted core coordinates
core = extracted_data['core']
start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])

# Crop the image to the region specified by the rectangle
cropped_image = image.copy()
cv2.rectangle(cropped_image, start_point, end_point, color, 2)
cropped_image = cropped_image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

# Display the original image
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()

plt.imshow(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()
# Display the cropped image




In [ ]:
# Save the cropped image
current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")
cv2.imwrite(f'extractor_single_img_test_{current_datetime}.png', cropped_image)


In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime

from pathlib import Path
def crop_fingerprint_core(image_path, output_folder):
    print(f"Processing image: {image_path}")
    output_file_name = os.path.splitext(os.path.basename(image_path))[0]+'.txt'

    if (Path(output_folder)/Path(output_file_name)).exists():
        return
    # check if input file is an image of ext tiff or bmp or tif
    if Path(image_path).suffix not in ['.tiff', '.bmp', '.tif']:
        print(f"Error reading image: {image_path}")
        return
    # Read the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error reading image: {image_path}")
        return
    
        
    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(image)
    # core = extracted_data['core']

    # Extract core coordinates
    # start_point = (core['x1'].values[0], core['y1'].values[0])
    # end_point = (core['x2'].values[0], core['y2'].values[0])

    # Crop the image to the region specified by the rectangle
    # cropped_image = image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # Resize the cropped image to 224x224
    # cropped_image = cv2.resize(cropped_image, (224, 224))

    # Save the resized image
    
    # output_path_image = os.path.join(output_folder, f'{output_file_name}.png')
    # cv2.imwrite(output_path_image, cropped_image)
    # print(f"Resized and cropped image saved: {output_path_image}")

    # Save minutiae information to a text file
    save_minutiae_to_text(output_file_name, extracted_data['minutiae'], output_folder)

def save_minutiae_to_text(file_name, minutiae_data, output_folder):
    output_text_file = os.path.join(output_folder, file_name)

    with open(output_text_file, 'w') as file:

        for idx, row in minutiae_data.iterrows():
            # Replace class with 1 if greater than 3
            # class_value = 1 if row['class'] > 3 else row['class']
            file.write(f"{row['x']} {row['y']} {row['angle']} {row['class']} {row['score']}\n")

def crop_fingerprint_cores_in_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each image in the input folder
    for _, _, files in os.walk(input_folder):
        for file_name in files:
            image_path = os.path.join(input_folder, file_name)
            crop_fingerprint_core(image_path, output_folder)

extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)



# Specify input and output folders
input_folder = '/home/rs/21CS91R01/research/our_FingerFlow/dataset/FVC2006DB2_A/'
output_folder = '/home/rs/21CS91R01/research/our_FingerFlow/dataset/croppedimages'

# Crop fingerprint cores for each image in the dataset
crop_fingerprint_cores_in_folder(input_folder, output_folder)


#for whole dataset

In [ ]:
import os
import cv2
from datetime import datetime

def crop_fingerprint_core(image_path, output_folder):
    # Read the image
    image = cv2.imread(image_path)

    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(image)
    core = extracted_data['core']

    # Extract core coordinates
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])

    # Crop the image to the region specified by the rectangle
    cropped_image = image.copy()
    cv2.rectangle(cropped_image, start_point, end_point, (0, 0, 255), 2)
    cropped_image = cropped_image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # Print the cropped image using OpenCV
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Print the dimensions of the cropped image
    print(f"Dimensions of the cropped image: {cropped_image.shape}")

    # Save the cropped image
    # current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    output_file_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path = os.path.join(output_folder, f'{output_file_name}.png')


   # output_path = os.path.join(output_folder, f'cropped_image_{current_datetime}.png')
    cv2.imwrite(output_path, cropped_image)

def crop_fingerprint_cores_in_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each image in the input folder
    for _, _, files in os.walk(input_folder):
        for file_name in files:
            image_path = os.path.join(input_folder, file_name)
            crop_fingerprint_core(image_path, output_folder)

# Initialize the extractor
extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

# Specify input and output folders
input_folder = '/content/drive/MyDrive/DB1_B/'
output_folder = '/content/drive/MyDrive/CroppedImages/'

# Crop fingerprint cores for each image in the dataset
crop_fingerprint_cores_in_folder(input_folder, output_folder)


#For datset roi as well as minutiae in .txt

In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime

def crop_fingerprint_core(image_path, output_folder):
    print(f"Processing image: {image_path}")

    # Read the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error reading image: {image_path}")
        return

    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(image)
    core = extracted_data['core']

    # Extract core coordinates
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])

    # Crop the image to the region specified by the rectangle
    cropped_image = image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # Resize the cropped image to 224x224
    cropped_image = cv2.resize(cropped_image, (224, 224))

    # Save the resized image
    output_file_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path_image = os.path.join(output_folder, f'{output_file_name}.png')
    cv2.imwrite(output_path_image, cropped_image)
    print(f"Resized and cropped image saved: {output_path_image}")

    # Save minutiae information to a text file
    save_minutiae_to_text(output_file_name, extracted_data['minutiae'], output_folder)

def save_minutiae_to_text(file_name, minutiae_data, output_folder):
    output_text_file = os.path.join(output_folder, f'{file_name}.txt')

    with open(output_text_file, 'w') as file:

        for idx, row in minutiae_data.iterrows():
            # Replace class with 1 if greater than 4
            class_value = 1 if row['class'] > 4 else row['class']
            file.write(f"{row['x']} {row['y']} {row['angle']} {class_value} {row['score']}\n")


# Specify input and output folders
input_folder = '/content/drive/MyDrive/DB1_B/'
output_folder = '/content/drive/MyDrive/CroppedImages/'

# Crop fingerprint cores for each image in the dataset
crop_fingerprint_cores_in_folder(input_folder, output_folder)


#Capacitve

In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime

def crop_fingerprint_core(image_path, output_folder):
    print(f"Processing image: {image_path}")

    # Read the image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error reading image: {image_path}")
        return

    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(image)
    core = extracted_data['core']

    # Extract core coordinates
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])

    # Crop the image to the region specified by the rectangle
    cropped_image = image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # Resize the cropped image to 224x224
    cropped_image = cv2.resize(cropped_image, (224, 224))

    # Save the resized image
    output_file_name = os.path.splitext(os.path.basename(image_path))[0]
    output_path_image = os.path.join(output_folder, f'{output_file_name}.png')
    cv2.imwrite(output_path_image, cropped_image)
    print(f"Resized and cropped image saved: {output_path_image}")

    # Save minutiae information to a text file
    save_minutiae_to_text(output_file_name, extracted_data['minutiae'], output_folder)

def save_minutiae_to_text(file_name, minutiae_data, output_folder):
    output_text_file = os.path.join(output_folder, f'{file_name}.txt')

    with open(output_text_file, 'w') as file:

        for idx, row in minutiae_data.iterrows():
            # Replace class with 1 if greater than 4
            class_value = 1 if row['class'] > 4 else row['class']
            file.write(f"{row['x']} {row['y']} {row['angle']} {class_value} {row['score']}\n")

def crop_fingerprint_cores_in_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each image in the input folder
    for _, _, files in os.walk(input_folder):
        for file_name in files:
            image_path = os.path.join(input_folder, file_name)
            crop_fingerprint_core(image_path, output_folder)

extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

# Specify input and output folders
input_folder = '/content/drive/MyDrive/DB3_B capacitive/'
output_folder = '/content/drive/MyDrive/CroppedImages_DB3_B capacitive'

# Crop fingerprint cores for each image in the dataset
crop_fingerprint_cores_in_folder(input_folder, output_folder)


#With 224*224 dimension resized roi

In [ ]:
import os
import cv2
from datetime import datetime
from google.colab.patches import cv2_imshow

def crop_fingerprint_core(image_path, output_folder, target_size=(224, 224)):
    # Read the image
    image = cv2.imread(image_path)

    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(image)
    core = extracted_data['core']

    # Extract core coordinates
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])

    # Crop the image to the region specified by the rectangle
    cropped_image = image[start_point[1]:end_point[1], start_point[0]:end_point[0]]

    # Resize the cropped image to the target size
    resized_image = cv2.resize(cropped_image, target_size)

    # Display the resized image using OpenCV
    cv2_imshow(resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Print the dimensions of the resized image
    print(f"Dimensions of the resized image: {resized_image.shape}")

    # Save the resized image with the original file name
    output_file_name = os.path.basename(image_path)
    output_path = os.path.join(output_folder, f'{output_file_name}.png')
    cv2.imwrite(output_path, resized_image)

def crop_fingerprint_cores_in_folder(input_folder, output_folder, target_size=(224, 224)):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each image in the input folder
    for _, _, files in os.walk(input_folder):
        for file_name in files:
            image_path = os.path.join(input_folder, file_name)
            crop_fingerprint_core(image_path, output_folder, target_size)

# Initialize the extractor
# extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)
extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

# Specify input and output folders
input_folder = '/content/drive/MyDrive/DB4_B/'
output_folder = '/content/drive/MyDrive/CroppedAndResizedImages/'

# Crop and resize fingerprint cores for each image in the dataset
crop_fingerprint_cores_in_folder(input_folder, output_folder)


#masking the roi


In [ ]:
import os
import cv2
import numpy as np
from datetime import datetime
from google.colab.patches import cv2_imshow

def mask_fingerprint_core(image_path, output_folder):
    # Read the image
    original_image = cv2.imread(image_path)

    # Extract minutiae data
    extracted_data = extractor.extract_minutiae(original_image)
    core = extracted_data['core']

    # Extract core coordinates
    start_point = (core['x1'].values[0], core['y1'].values[0])
    end_point = (core['x2'].values[0], core['y2'].values[0])

    # Create a binary mask for the region of interest
    mask = np.zeros_like(original_image)
    cv2.rectangle(mask, start_point, end_point, (255, 255, 255), thickness=cv2.FILLED)

    # Apply the mask to the original image
    masked_image = cv2.bitwise_and(original_image, mask)

    # Display the masked image using OpenCV
    cv2_imshow(masked_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Print the dimensions of the masked image
    print(f"Dimensions of the masked image: {masked_image.shape}")

    # Save the masked image
    current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    output_path = os.path.join(output_folder, f'masked_image_{current_datetime}.png')
    cv2.imwrite(output_path, masked_image)

def mask_fingerprint_cores_in_folder(input_folder, output_folder):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Process each image in the input folder
    for _, _, files in os.walk(input_folder):
        for file_name in files:
            image_path = os.path.join(input_folder, file_name)
            mask_fingerprint_core(image_path, output_folder)

# Initialize the extractor
#extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

# Specify input and output folders
input_folder = '/content/drive/MyDrive/DB4_B/'
output_folder = '/content/drive/MyDrive/MaskedImages/'

# Mask fingerprint cores for each image in the dataset
mask_fingerprint_cores_in_folder(input_folder, output_folder)


#Not useful for now

In [ ]:
from datetime import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
# from /content/drive/MyDrive/fingerflow-main/src/fingerflow/extractor/extractor.py
# # from fingerflow import extractor
# from fingerflow import extractor
# from .content.drive.MyDrive.fingerflow_main.src.fingerflow.extractor.extractor import *
# from fingerflow.extractor.CoreNet import *
# from fingerflow.extractor.extractor import Extractor
# from fingerflow.extractor.extractor import *


COARSE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/CoarseNet.h5"
FINE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/FineNet.h5"
CLASSIFY_NET_PATH = "/content/drive/MyDrive/fingerflow-main/ClassifyNet_6_classes.h5"
CORE_NET_PATH = '/content/drive/MyDrive/fingerflow-main/CoreNet.weights'
IMG_PATH = '/content/101_2.tif'
RADIUS = 5


def get_minutiae_color(point_class):
    switcher = {
        0: (0, 0, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")

extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

image = cv2.imread(IMG_PATH)

extracted_data = extractor.extract_minutiae(image)

extracted_minutiae = extracted_data['minutiae']
core = extracted_data['core']

start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])
color = (0, 0, 255)

cv2.rectangle(image, start_point, end_point, color, 2)


if len(extracted_minutiae) > 0:
    for index, minutia in extracted_minutiae.iterrows():
        print(minutia)
        x = minutia['x']
        y = minutia['y']
        o = minutia['angle']
        minutia_class = minutia['class']

        start_point = (int(x-RADIUS), int(y-RADIUS))
        end_point = (int(x+RADIUS), int(y+RADIUS))

        color = get_minutiae_color(int(minutia_class))

        cv2.rectangle(image, start_point, end_point, color, 2)

        line_start_point = (int(x), int(y))
        line_end_point = (int(x + 20 * np.cos(o)), int(y + 20 * np.sin(o)))

        cv2.line(image, line_start_point, line_end_point, color, 2)

cv2_imshow( image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(f'extractor_single_img_test_{current_datetime}.png', image)


In [ ]:
from datetime import datetime
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
# from /content/drive/MyDrive/fingerflow-main/src/fingerflow/extractor/extractor.py
# # from fingerflow import extractor
# from fingerflow import extractor
# from .content.drive.MyDrive.fingerflow_main.src.fingerflow.extractor.extractor import *
# from fingerflow.extractor.CoreNet import *
# from fingerflow.extractor.extractor import Extractor
# from fingerflow.extractor.extractor import *


COARSE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/CoarseNet.h5"
FINE_NET_PATH = "/content/drive/MyDrive/fingerflow-main/FineNet.h5"
CLASSIFY_NET_PATH = "/content/drive/MyDrive/fingerflow-main/ClassifyNet_6_classes.h5"
CORE_NET_PATH = '/content/drive/MyDrive/fingerflow-main/CoreNet.weights'
IMG_PATH = '/content/104_4.tif'
RADIUS = 5


def get_minutiae_color(point_class):
    switcher = {
        0: (0, 0, 255),
        1: (0, 255, 0),
        2: (255, 0, 0),
        3: (0, 255, 255),
        4: (255, 255, ),
        5: (255, 0, 255)
    }

    return switcher.get(point_class)


current_datetime = datetime.now().strftime("%d%m%Y%H%M%S")

#extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

image = cv2.imread(IMG_PATH)

extracted_data = extractor.extract_minutiae(image)

extracted_minutiae = extracted_data['minutiae']
core = extracted_data['core']

start_point = (core['x1'].values[0], core['y1'].values[0])
end_point = (core['x2'].values[0], core['y2'].values[0])
color = (0, 0, 255)

cv2.rectangle(image, start_point, end_point, color, 2)


if len(extracted_minutiae) > 0:
    for index, minutia in extracted_minutiae.iterrows():
        print(minutia)
        x = minutia['x']
        y = minutia['y']
        o = minutia['angle']
        minutia_class = minutia['class']

        start_point = (int(x-RADIUS), int(y-RADIUS))
        end_point = (int(x+RADIUS), int(y+RADIUS))

        color = get_minutiae_color(int(minutia_class))

        cv2.rectangle(image, start_point, end_point, color, 2)

        line_start_point = (int(x), int(y))
        line_end_point = (int(x + 20 * np.cos(o)), int(y + 20 * np.sin(o)))

        cv2.line(image, line_start_point, line_end_point, color, 2)

cv2_imshow( image)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite(f'extractor_single_img_test_{current_datetime}.png', image)


In [ ]:
# import pickle
# import cv2
# import numpy as np
# import pandas as pd
# # from fingerflow import extractor

# # COARSE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/CoarseNet.h5"
# # FINE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/FineNet.h5"
# # CLASSIFY_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/ClassifyNet.h5"
# # CORE_NET_PATH = '/home/jakub/projects/dp/kernel-detector/trainings/19-12-2021-final/yolo-kernel_best.weights'

# IMG_PATH = '/home/user/Downloads/fingerflow-main/scripts/extractor/102_4.tif'
# DATA_PATH = 'extractor_output.txt'

# MINUTIAE_NUM = 20

# # extractor = extractor.Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)

# image = cv2.imread(IMG_PATH)

# extracted_minutiae = extractor.extract_minutiae(image)

# with open(DATA_PATH, 'wb') as f:
#     pickle.dump(extracted_minutiae, f)


# def get_correct_core_point(core_data):
#     correct_core = core_data[core_data.score == core_data.score.max()]

#     x = correct_core[['x1', 'x2']].mean(axis=1)
#     y = correct_core[['y1', 'y2']].mean(axis=1)

#     return pd.DataFrame({'x': x.values, 'y': y.values})


# def get_n_nearest_minutiae(extracted_data, core_point):
#     minutiae_data = extracted_data['minutiae']

#     if len(core_point) == 0 or len(minutiae_data) < MINUTIAE_NUM:
#         return pd.DataFrame()

#     minutiae_data['core_distance'] = np.linalg.norm(
#         minutiae_data[['x', 'y']].values - core_point.values, axis=1)

#     return minutiae_data.sort_values(by=['core_distance']).reset_index(drop=True)


# def get_minutiae_color(point_class):
#     switcher = {
#         0: (255, 255, 255),
#         1: (0, 255, 0),
#         2: (255, 0, 0),
#         3: (0, 255, 255),
#         4: (255, 255, ),
#         5: (255, 0, 255)
#     }

#     return switcher.get(point_class)


# def show_core_point(core):
#     start_point = (core['x1'].values[0], core['y1'].values[0])
#     end_point = (core['x2'].values[0], core['y2'].values[0])
#     color = (0, 0, 255)

#     cv2.rectangle(image, start_point, end_point, color, 2)


# def show_minutiae_points(minutiae, core_point):
#     radius = 10
#     core = (int(core_point['x']), int(core_point['y']))

#     for index, point in minutiae.iterrows():
#         center = (int(point['x']), int(point['y']))

#         print(center, index)
#         cv2.circle(image, center, radius, get_minutiae_color(int(point['class'])), 1)

#         if index < MINUTIAE_NUM:
#             cv2.line(image, core, center, (0, 0, 255), 1)


# with open(DATA_PATH, 'rb') as f:
#     extracted_minutiae = pickle.load(f)

#     core_point = get_correct_core_point(extracted_minutiae['core'])
#     enriched_minutiae = get_n_nearest_minutiae(extracted_minutiae, core_point)

#     show_core_point(extracted_minutiae['core'])
#     show_minutiae_points(enriched_minutiae, core_point)

#     print(enriched_minutiae.to_markdown())
#     cv2.imshow('fingerpint', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

#     cv2.imwrite('visualised_feature_vector_3.png', image)


In [ ]:
# import os
# import cv2
# import numpy as np
# # USING PYPI PACKAGE
# # from fingerflow import extractor
# # USING LOCAL VERSION - move script ro parent folder to run it
# # from fingerflow import extractor

# # COARSE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/CoarseNet.h5"
# # FINE_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/FineNet.h5"
# # CLASSIFY_NET_PATH = "/home/jakub/projects/bp/minutiae_classificator/models/ClassifyNet.h5"
# # CORE_NET_PATH = '/home/jakub/projects/dp/kernel-detector/trainings/19-12-2021-final/yolo-kernel_best.weights'
# # COARSE_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/CoarseNet.h5"
# # FINE_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/FineNet.h5"
# # CLASSIFY_NET_PATH = "/home/user/Downloads/fingerflow-main/scripts/extractor/ClassifyNet_6_classes.h5"
# # CORE_NET_PATH = '/home/user/Downloads/fingerflow-main/scripts/extractor/CoreNet.weights'

# # extractor = Extractor(COARSE_NET_PATH, FINE_NET_PATH, CLASSIFY_NET_PATH, CORE_NET_PATH)


# def read_image(image_path):
#     original_image = np.array(cv2.imread(image_path, 0))
#     image_size = np.array(original_image.shape, dtype=np.int32) // 8 * 8
#     image = original_image[:image_size[0], :image_size[1]]

#     output = dict()

#     output['original_image'] = original_image
#     output['image_size'] = image_size
#     output['image'] = image

#     return output


# def get_extracted_minutiae_data(image_path):
#     image = np.array(cv2.imread(image_path, 0))

#     extracted_minutiae = extractor.extract_minutiae(image)

#     return extracted_minutiae


# def get_extracted_minutiae(image_folder):
#     minutiae_files = []

#     for _, _, files in os.walk(image_folder):
#         for file_name in files:
#             file_path = image_folder + file_name

#             minutiae_data = get_extracted_minutiae_data(file_path)
#             print(f"{file_name} - processed => ")
#             print(minutiae_data)

#     #         file_name_without_extension = os.path.splitext(os.path.basename(file_name))[0]
#     #         minutiae = Minutiae(file_name_without_extension, minutiae_data)
#     #         minutiae_files.append(minutiae)
#     #         print(minutiae_files)

#     # return minutiae_files


# fil=get_extracted_minutiae(
#     '/content/drive/MyDrive/fingerflow-main/tests/DB1_B/')


#GEtting Vectorssss from dataset

In [ ]:
# def read_image(image_path):
#     original_image = np.array(cv2.imread(image_path,cv2.IMREAD_UNCHANGED)
#     image_size = np.array(original_image.shape, dtype=np.int32) // 8 * 8
#     image = original_image[:image_size[0], :image_size[1]]

#     output = dict()

#     output['original_image'] = original_image
#     output['image_size'] = image_size
#     output['image'] = image

#     return output
import os
import cv2
import numpy as np
minutiae_pts=[]
core_pts=[]

def get_extracted_minutiae_data(image_path):


    image = np.array(cv2.imread(image_path))

    extracted_minutiae = extractor.extract_minutiae(image)

    return extracted_minutiae


def get_extracted_minutiae(image_folder):
    # aftab = []


    for _, _, files in os.walk(image_folder):
        for file_name in files:
            file_path = image_folder + file_name
            # print(file_path)

            # image = np.array(cv2.imread(file_path))
            # height, width, channels = image.shape
            # print(channels)

            minutiae_data = get_extracted_minutiae_data(file_path)
            print(minutiae_data)

            #minutiae_pts.append(minutiae_data['minutiae'].values)
            #core_pts.append(minutiae_data['core'].values)
            # print(minutiae_pts)
            # print(f"{file_name} - processed => ")
            # print(minutiae_data)

    #         file_name_without_extension = os.path.splitext(os.path.basename(file_name))[0]
    #         minutiae = Minutiae(file_name_without_extension, minutiae_data)
    #         minutiae_files.append(minutiae)
    #         print(minutiae_files)

    # return minutiae_files


get_extracted_minutiae(
    '/content/drive/MyDrive/DB4_B/')

In [ ]:
core_pts = minutiae_data['core'].values
minutiae_pts = minutiae_data['minutiae'].values
print(minutiae_pts)

In [ ]:
import os
import cv2
import numpy as np
minutiae_pts = []
core_pts = []

def get_extracted_minutiae_data(image_path):
    image = np.array(cv2.imread(image_path))
    extracted_minutiae = extractor.extract_minutiae(image)
    return extracted_minutiae

def get_extracted_minutiae(image_folder):
    minutiae_data_list = []

    for _, _, files in os.walk(image_folder):
        for file_name in files:
            file_path = image_folder + file_name

            minutiae_data = get_extracted_minutiae_data(file_path)
            minutiae_data_list.append(minutiae_data)

    return minutiae_data_list

minutiae_data_list = get_extracted_minutiae('/content/drive/MyDrive/DB4_B/')

# Access the minutiae data for each image
for minutiae_data in minutiae_data_list:
    print(minutiae_data)
    core_pts = minutiae_data['core'].values
    minutiae_pts = minutiae_data['minutiae'].values



In [ ]:
core_pts_list = []
minutiae_pts_list = []



# Access the minutiae data for each image
for minutiae_data in minutiae_data_list:
    print(minutiae_data)
    core_pts = minutiae_data['core'].values
    minutiae_pts = minutiae_data['minutiae'].values
    core_pts_list.append(core_pts)
    minutiae_pts_list.append(minutiae_pts)


In [ ]:
minutiae_pts_list

In [ ]:
import pandas as pd
# Access the minutiae data for each image
for minutiae_data in minutiae_data_list:

  core = minutiae_data['core']
  minutiae = minutiae_data['minutiae']

# Filter minutiae based on core points
  filtered_minutiae = minutiae[
      (minutiae['x'] >= core['x1'].values[0]) &
      (minutiae['x'] <= core['x2'].values[0]) &
      (minutiae['y'] >= core['y1'].values[0]) &
      (minutiae['y'] <= core['y2'].values[0])
  ]

  print(filtered_minutiae)


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the data frames
df1 = pd.DataFrame({
    'x': [219.0, 209.0, 194.0, 181.0, 193.0, 210.0],
    'y': [139.0, 114.0, 132.0, 211.0, 182.0, 154.0],
    'angle': [1.292227, 0.677497, 2.155247, 1.258612, 1.276576, 1.273878]
})

df2 = pd.DataFrame({
    'x': [259.0, 238.0, 242.0, 218.0, 226.0, 267.0],
    'y': [210.0, 189.0, 218.0, 253.0, 204.0, 274.0],
    'angle': [1.218769, 2.661162, 1.405085, 1.390800, 2.405977, 4.315348]
})

# Concatenate the two data frames
concatenated = pd.concat([df1, df2])

# Calculate the pairwise cosine similarity
similarity_matrix = cosine_similarity(concatenated)

# Extract the similarity values for the two data frames
similarity_values = similarity_matrix[:len(df1), len(df1):]

# Print the similarity values
print(similarity_values)



In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Define the original data frame
df1 = pd.DataFrame({
    'x': [219.0, 209.0, 194.0, 181.0, 193.0, 210.0],
    'y': [139.0, 114.0, 132.0, 211.0, 182.0, 154.0],
    'angle': [1.292227, 0.677497, 2.155247, 1.258612, 1.276576, 1.273878]
})

# Define the updated data frame
df2 = pd.DataFrame({
    'x': [132.0, 158.0, 156.0, 122.0, 115.0, 147.0],
    'y': [84.0, 53.0, 82.0, 124.0, 106.0, 107.0],
    'angle': [0.596588, 0.942346, 3.644766, 0.717803, 0.629055, 3.043717]
})

# Concatenate the two data frames
concatenated = pd.concat([df1, df2])

# Calculate the pairwise cosine similarity
similarity_matrix = cosine_similarity(concatenated)

# Extract the similarity values for the two data frames
similarity_values = similarity_matrix[:len(df1), len(df1):]

# Print the similarity values
print(similarity_values)


In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def calculate_minutiae_similarity(minutiae_pts_1, minutiae_pts_2):
    # Calculate similarity between minutiae points

    # Extract the coordinates of the minutiae points
    coords_1 = minutiae_pts_1[:, :2]
    coords_2 = minutiae_pts_2[:, :2]

    # Calculate the cosine distance matrix between the minutiae coordinates
    distance_matrix = 1 - np.dot(coords_1, coords_2.T) / (
        np.linalg.norm(coords_1, axis=1)[:, np.newaxis] *
        np.linalg.norm(coords_2, axis=1)
    )

    # Use the Hungarian algorithm to find the best matching between the points
    row_ind, col_ind = linear_sum_assignment(distance_matrix)

    # Calculate the total distance of the best matching points
    total_distance = distance_matrix[row_ind, col_ind].sum()

    # Calculate the similarity score based on the inverse of the total distance
    similarity_score = 1 / (1 + total_distance)

    return similarity_score

In [ ]:
import pandas as pd
import seaborn as sns
from scipy.spatial import distance

# Calculate the pairwise distances for 'x', 'y', and 'angle' fields
data = minutiae_data_list
dist_matrix = distance.cdist(data, data, metric='euclidean')

# Create a heatmap using Seaborn
sns.heatmap(dist_matrix, annot=True, cmap='viridis')

# Show the plot
plt.show()


In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Define the data points
data1 = pd.DataFrame({
    'x': [219.0, 209.0, 194.0, 181.0, 193.0, 210.0],
    'y': [139.0, 114.0, 132.0, 211.0, 182.0, 154.0],
    'angle': [1.292227, 0.677497, 2.155247, 1.258612, 1.276576, 1.273878],
    'score': [0.905982, 0.979381, 0.785645, 0.760999, 0.776544, 0.582125],
    'class': [5.0, 5.0, 5.0, 5.0, 5.0, 0.0]
})

data2 = pd.DataFrame({
    'x': [259.0, 238.0, 242.0, 218.0, 226.0, 267.0],
    'y': [210.0, 189.0, 218.0, 253.0, 204.0, 274.0],
    'angle': [1.218769, 2.661162, 1.405085, 1.390800, 2.405977, 4.315348],
    'score': [0.939920, 0.770226, 0.772666, 0.760236, 0.734271, 0.779438],
    'class': [0.0, 5.0, 5.0, 5.0, 5.0, 5.0]
})

# Select the 'x', 'y', and 'angle' fields
data1_selected = data1[['x', 'y', 'angle']]
data2_selected = data2[['x', 'y', 'angle']]

# Calculate the cosine similarity
similarity_matrix = cosine_similarity(data1_selected, data2_selected)

# Convert similarity matrix to DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=data1.index, columns=data2.index)

# Show the similarity matrix
print(similarity_df)



In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

def get_extracted_minutiae_data(image_path):
    image = np.array(cv2.imread(image_path))
    extracted_minutiae = extractor.extract_minutiae(image)
    return extracted_minutiae

def get_extracted_minutiae(image_folder):
    minutiae_data_list = []

    for _, _, files in os.walk(image_folder):
        for file_name in files:
            file_path = image_folder + file_name

            minutiae_data = get_extracted_minutiae_data(file_path)
            minutiae_data_list.append(minutiae_data)

    return minutiae_data_list

def filter_minutiae_by_core(minutiae_data, core_data):
    filtered_minutiae = []
    core_x1, core_y1, core_x2, core_y2 = core_data[['x1', 'y1', 'x2', 'y2']].values[0]

    for image_data in minutiae_data:
        minutiae_points = image_data['minutiae']
        filtered_points = minutiae_points[
            (minutiae_points['x'] >= core_x1) &
            (minutiae_points['x'] <= core_x2) &
            (minutiae_points['y'] >= core_y1) &
            (minutiae_points['y'] <= core_y2)
        ]
        filtered_minutiae.append(filtered_points.values)  # Append values instead of the DataFrame itself

    return filtered_minutiae

# Assuming you have the minutiae data for each image stored in minutiae_data_list
minutiae_data_list = get_extracted_minutiae('/content/drive/MyDrive/DB4_B/')

# Assuming you have the core data for each image stored in core_data_list
core_data_list = []

# Access the minutiae and core data for each image
for minutiae_data, core_data in zip(minutiae_data_list, core_data_list):
    filtered_minutiae = filter_minutiae_by_core(minutiae_data, core_data)
    for points in filtered_minutiae:
        print(points)


In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def calculate_minutiae_similarity(minutiae_pts_1, minutiae_pts_2):
    # Calculate similarity between minutiae points

    # Extract the coordinates of the minutiae points
    coords_1 = minutiae_pts_1[:, :2]
    coords_2 = minutiae_pts_2[:, :2]

    # Calculate the cosine distance matrix between the minutiae coordinates
    distance_matrix = 1 - np.dot(coords_1, coords_2.T) / (
        np.linalg.norm(coords_1, axis=1)[:, np.newaxis] *
        np.linalg.norm(coords_2, axis=1)
    )

    # Use the Hungarian algorithm to find the best matching between the points
    row_ind, col_ind = linear_sum_assignment(distance_matrix)

    # Calculate the total distance of the best matching points
    total_distance = distance_matrix[row_ind, col_ind].sum()

    # Calculate the similarity score based on the inverse of the total distance
    similarity_score = 1 / (1 + total_distance)

    return similarity_score

In [ ]:
def check_similarity(fingerprint1, fingerprint2, core_weight, minutiae_weight, core_threshold, minutiae_threshold):
    # Compare core points
    if len(fingerprint1['core']) != len(fingerprint2['core']):
        return False
    core_similarity = calculate_core_similarity(fingerprint1['core'], fingerprint2['core'])
    if core_similarity < core_threshold:
        return False

    # Compare minutiae points
    minutiae_similarity = calculate_minutiae_similarity(fingerprint1['minutiae'], fingerprint2['minutiae'])
    if minutiae_similarity < minutiae_threshold:
        return False

    # Combine core and minutiae similarity scores
    similarity_score = (core_similarity * core_weight) + (minutiae_similarity * minutiae_weight)

    # Check if the combined similarity score is above the threshold
    if similarity_score >= similarity_threshold:
        return True
    else:
        return False

# Example usage
fingerprint1 = {
    'core': core_points1,     # DataFrame of core points
    'minutiae': minutiae_points1     # DataFrame of minutiae points
}

fingerprint2 = {
    'core': core_points2,     # DataFrame of core points
    'minutiae': minutiae_points2     # DataFrame of minutiae points
}

core_weight = 0.5
minutiae_weight = 0.5
core_threshold = 0.8
minutiae_threshold = 0.6
similarity_threshold = 0.7

is_similar = check_similarity(fingerprint1, fingerprint2, core_weight, minutiae_weight, core_threshold, minutiae_threshold)
print("Are the fingerprints similar?", is_similar)


#Dont runnnnn itt unless for A NEW DATASET

In [ ]:
# import os
# import pickle

# # Directory to save the pickle files
# output_dir = '/content/drive/MyDrive'

# # Create the output directory if it doesn't exist
# os.makedirs(output_dir, exist_ok=True)

# # Save the minutiae_pts list to a file
# minutiae_pts_file = os.path.join(output_dir, 'minutiae_pts.pkl')
# with open(minutiae_pts_file, 'wb') as f:
#     pickle.dump(minutiae_pts, f)

# # Save the core_pts list to a file
# core_pts_file = os.path.join(output_dir, 'core_pts.pkl')
# with open(core_pts_file, 'wb') as f:
#     pickle.dump(core_pts, f)


In [ ]:
import os
import pickle

# Directory where the pickle files are saved
output_dir = '/content/drive/MyDrive'

# Load the minutiae_pts list from the file
minutiae_pts_file = os.path.join(output_dir, 'minutiae_pts.pkl')
with open(minutiae_pts_file, 'rb') as f:
    minutiae_pts = pickle.load(f)

# Load the core_pts list from the file
core_pts_file = os.path.join(output_dir, 'core_pts.pkl')
with open(core_pts_file, 'rb') as f:
    core_pts = pickle.load(f)


In [ ]:


print((core_pts[0][0][0]))
# # print(core_pts)
print(minutiae_pts[1])
print(type(core_pts))

In [ ]:
def compare_minutiae_vectors(vector_a, vector_b, threshold):
    match_count = 0
    total_minutiae = len(vector_a)

    for i in range(total_minutiae):
        x_a, y_a, angle_a = vector_a[i]

        for j in range(total_minutiae):
            x_b, y_b, angle_b = vector_b[j]

            dx = abs(x_a - x_b)
            dy = abs(y_a - y_b)
            d_angle = abs(angle_a - angle_b)

            if dx <= threshold and dy <= threshold and d_angle <= threshold:
                match_count += 1
                break

    matching_percentage = (match_count / total_minutiae) * 100

    if matching_percentage >= threshold:
        return True
    else:
        return False


In [ ]:
def calculate_core_similarity(core_pts_1, core_pts_2):
    # Calculate similarity between core points

    # Extract the core bounding boxes
    box_1 = core_pts_1[:, :4]
    box_2 = core_pts_2[:, :4]

    # Calculate the intersection area
    intersection_area = np.maximum(0, np.minimum(box_1[:, 2], box_2[:, 2]) - np.maximum(box_1[:, 0], box_2[:, 0])) * \
                        np.maximum(0, np.minimum(box_1[:, 3], box_2[:, 3]) - np.maximum(box_1[:, 1], box_2[:, 1]))

    # Calculate the union area
    union_area = (box_1[:, 2] - box_1[:, 0]) * (box_1[:, 3] - box_1[:, 1]) + \
                 (box_2[:, 2] - box_2[:, 0]) * (box_2[:, 3] - box_2[:, 1]) - \
                 intersection_area

    # Calculate the IoU score
    iou_score = intersection_area / union_area

    # Take the maximum IoU score across all bounding boxes
    core_similarity = np.max(iou_score)

    return core_similarity



In [ ]:
calculate_core_similarity(core_pts[1], core_pts[2])

In [ ]:
!pip install fingerflow

In [ ]:
from fingerflow.matcher import Matcher


In [ ]:
from fingerflow.extractor import constants
import fingerflow

In [ ]:
from fingerflow.matcher import verify_net




In [ ]:
from fingerflow.matcher.VerifyNet import verify_net_model


verify_net

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend
from fingerflow.matcher.VerifyNet import verify_net_model
from fingerflow.matcher.VerifyNet import utils



class VerifyNet:
    def __init__(self, precision, verify_net_path):
        backend.clear_session()

        self.__verify_net = verify_net_model.get_verify_net_model(precision, verify_net_path)

    def verify_fingerprints(self, anchor, sample):
        preprocessed_input = utils.preprocess_predict_input(anchor, sample)

        # change according to docs -> https://keras.io/api/models/model_training_apis/#predict-method
        [[prediction]] = self.__verify_net.predict(preprocessed_input)

        return prediction

    def verify_fingerprints_batch(self, pairs):
        preprocessed_pairs = [utils.preprocess_predict_input(
            anchor, sample) for [anchor, sample] in pairs]

        def predict_item(pair):
            [[prediction]] = self.__verify_net.predict(pair)

            return prediction

        predictions = [predict_item(pair) for pair in preprocessed_pairs]

        return predictions
        # TODO : find out how this can work :D
        # return self.__verify_net.predict(
        #     np.vstack((preprocessed_pairs[0],
        #               preprocessed_pairs[1])))

    def plot_model(self, file_path):
        tf.keras.utils.plot_model(self.__verify_net, to_file=file_path,
                                  show_shapes=True, expand_nested=True)


matcher.py

In [ ]:
# from .verify_net import VerifyNet


class Matcher:
    def __init__(self, precision, verify_net_path):
        self.__verification_module = VerifyNet(precision, verify_net_path)

    def verify(self, anchor, sample):
        return self.__verification_module.verify_fingerprints(anchor, sample)

    def verify_batch(self, pairs):
        return self.__verification_module.verify_fingerprints_batch(pairs)

    def plot_model(self, file_path):
        self.__verification_module.plot_model(file_path)


In [ ]:
#kehiecore_pts_list
core_pts = core_pts_list[0]
minutiae_pts=minutiae_pts_list[0]


print(minutiae_pts)



In [ ]:


# Extract the core coordinates from core_pts
core_x1 = core_pts[0][0]
core_y1 = core_pts[0][1]
core_x2 = core_pts[0][2]
core_y2 = core_pts[0][3]

# Extract the minutiae coordinates from minutiae_pts
minutiae_coordinates = np.array([minutiae[:2] for minutiae in minutiae_pts])

# Calculate the distance between each minutia point and the core point
distances_to_core = np.linalg.norm(minutiae_coordinates - np.array([core_x1, core_y1]), axis=1)

# Add the distances to the core as an additional column in minutiae_pts
minutiae_pts_with_distance = np.column_stack((minutiae_pts, distances_to_core))

# Use the matching feature vectors method with the modified minutiae_pts_with_distance
matching_confidence = (minutiae_pts_with_distance)

# Create a new array with the desired shape and assign values
matching_confidence_with_zeros = np.zeros((matching_confidence.shape[0] + 6, matching_confidence.shape[1]))
matching_confidence_with_zeros[:matching_confidence.shape[0]] = matching_confidence
# Print the matching confidence
print(matching_confidence_with_zeros)


In [ ]:


# Extract the core coordinates from core_pts
core_x1 = cp[0][0]
core_y1 = cp[0][1]
core_x2 = cp[0][2]
core_y2 = cp[0][3]

# Extract the minutiae coordinates from minutiae_pts
minutiae_coordinates = np.array([minutiae[:2] for minutiae in mp])

# Calculate the distance between each minutia point and the core point
distances_to_core = np.linalg.norm(minutiae_coordinates - np.array([core_x1, core_y1]), axis=1)

# Add the distances to the core as an additional column in minutiae_pts
minutiae_pts_with_distance = np.column_stack((mp, distances_to_core))

# Use the matching feature vectors method with the modified minutiae_pts_with_distance
matching_confidence1 = (minutiae_pts_with_distance)

# Create a new array with the desired shape and assign values
matching_confidence_with_zeros1 = np.zeros((matching_confidence1.shape[0] + 4, matching_confidence1.shape[1]))
matching_confidence_with_zeros1[:matching_confidence1.shape[0]] = matching_confidence1
# Print the matching confidence
print(len(matching_confidence_with_zeros1))

In [ ]:
from fingerflow.matcher import Matcher

matcher = Matcher(30, "/content/drive/MyDrive/fingerflow-main/VerfiyNet-30.h5")

prediction = matcher.verify(matching_confidence_with_zeros1, matching_confidence_with_zeros)

In [ ]:
print(prediction)

In [ ]:
import numpy as np

md = {
    'core': {
        'x1': 175,
        'y1': 91,
        'x2': 233,
        'y2': 213,
        'score': 0.938329,
        'w': 58,
        'h': 122
    },
    'minutiae': {
        'x': [219.0, 77.0, 57.0, 106.0, 100.0, 234.0, 153.0, 82.0, 82.0, 243.0, 81.0, 147.0, 94.0, 209.0, 194.0, 121.0, 181.0, 81.0, 193.0, 101.0, 174.0, 210.0, 91.0, 237.0, 50.0, 147.0],
        'y': [139.0, 125.0, 193.0, 221.0, 123.0, 149.0, 204.0, 177.0, 214.0, 129.0, 154.0, 61.0, 245.0, 114.0, 132.0, 105.0, 211.0, 321.0, 182.0, 204.0, 331.0, 154.0, 300.0, 187.0, 285.0, 82.0],
        'angle': [1.292227, 2.23079, 2.138606, 1.602496, 2.208304, 1.215875, 1.381653, 2.051182, 1.675593, 1.013333, 2.16016, 2.656905, 1.394019, 0.677497, 2.155247, 2.200649, 1.258612, 4.004857, 1.276576, 1.980142, 4.036953, 1.273878, 0.667045, 1.253688, 0.678109, 2.310249],
        'score': [0.905982, 0.800195, 0.79942, 0.7994, 0.952884, 0.798649, 0.798314, 0.796859, 0.796118, 0.795029, 0.987752, 0.791158, 0.789956, 0.979381, 0.785645, 0.924349, 0.760999, 0.927959, 0.776544, 0.733349, 0.696058, 0.582125, 0.581615, 0.527384, 0.518953, 0.471133],
        'class': [5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 0.0, 5.0, 5.0, 5.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 0.0, 2.0, 5.0, 4.0, 5.0]
    }
}

mp = np.array([
    md['minutiae']['x'],
    md['minutiae']['y'],
    md['minutiae']['angle'],
    md['minutiae']['score'],
    md['minutiae']['class']
])

cp = np.array([
    [
        md['core']['x1'],
        md['core']['y1'],
        md['core']['x2'],
        md['core']['y2'],
        md['core']['score'],
        md['core']['w'],
        md['core']['h']
    ]
])
mp = mp.T
print("minutiae_pts:\n", mp)
print("core_pts:\n", cp)
